# Spotify elemzésem – 8 alap lépés + 5 extra lekérdezés (összesen 13)

Ez az én egyszerű, átlátható notebookom. Lépésről lépésre haladok:

1) Könyvtárak betöltése
2) Adatok beolvasása (JSON fájlok egyesítése)
3) Minimális előkészítés (dátum, perc, egyszerű címkék)

Lekérdezések:

4) Összes hallgatási idő év/hónap szerint

5) Top 10 előadó (perc) és Top 10 dal (lejátszásszám)

6) Időbeli szokások: hét napja × óra pivot + offline/online arány

7) Platform-megoszlás (összes perc és részesedés %)

8) Top 10 nap – legtöbb hallgatott perc + aktív napok száma

9) Havi előadói diverzitás – egyedi előadók száma hónaponként

10) Hosszú lejátszások aránya (≥ 2 perc) – összesen és havi bontásban

11) Óránkénti megoszlás – melyik órákban hallgatok legtöbbet

12) Napok szerinti megoszlás – melyik hétköznapokon hallgatok legtöbbet

13) Átugrási ráta előadónként (mintaszűrés mellett)


In [97]:
# 1) Könyvtárak
import json  # JSON fájlok beolvasásához
from pathlib import Path  # Platformfüggetlen fájl-/útvonalkezeléshez
import numpy as np  # Numerikus műveletekhez (NaN, százalékok, stb.)
import pandas as pd  # Adatkezeléshez és -elemzéshez (DataFrame)
from IPython.display import display  # Táblázatok szép megjelenítéséhez notebookban

# Szebb táblázat-kimenet (pandas megjelenítési beállítások)
pd.set_option('display.max_rows', 50)  # Max. megjelenített sorok száma
pd.set_option('display.max_columns', 50)  # Max. megjelenített oszlopok száma
pd.set_option('display.width', 120)  # Kimenet szélessége karakterben

In [98]:
# 2) Adatok beolvasása (egy mappából több JSON egyesítése)
DATA_DIR = Path.cwd() / 'Spotify_Extended_Streaming_History'  # Spotify adatok mappája (állítsd át, ha máshol van)
AUDIO_PAT = 'Streaming_History_Audio_*.json'  # Audio JSON fájlok mintája
VIDEO_PAT = 'Streaming_History_Video_*.json'  # Video JSON fájlok mintája (opcionális)

def read_json_list(p: Path) -> list:
    with p.open('r', encoding='utf-8') as f:  # Fájl megnyitása olvasásra, UTF-8 kódolással
        return json.load(f)  # JSON betöltése (listát várunk)

rows = []  # Ide gyűjtöm az összes rekordot minden fájlból
for p in sorted(DATA_DIR.glob(AUDIO_PAT)):  # Audio fájlok bejárása, név szerint rendezve
    try:
        rows.extend(read_json_list(p))  # A fájlban lévő listát hozzáfűzöm a gyűjtőlistához
    except Exception as e:
        print(f'Hiba audio fájl olvasásakor: {p.name} -> {e}')  # Hiba esetén fájlnév + hiba kiírása
for p in sorted(DATA_DIR.glob(VIDEO_PAT)):  # Video fájlok bejárása, név szerint rendezve
    try:
        rows.extend(read_json_list(p))  # Hozzáfűzöm a videós rekordokat is (ha vannak)
    except Exception as e:
        print(f'Hiba video fájl olvasásakor: {p.name} -> {e}')  # Hiba esetén logolás

df = pd.DataFrame(rows)  # Az összes rekordot táblába teszem (további elemzéshez)
print(f'Összes sor: {len(df):,}')  # Összes sor száma, ezres elválasztóval
display(df.head(3))  # Minta: első 3 sor megjelenítése ellenőrzésképp

Összes sor: 134,514


,ts,platform,ms_played,conn_country,ip_addr,master_metadata_track_name,master_metadata_album_artist_name,master_metadata_album_album_name,spotify_track_uri,episode_name,episode_show_name,spotify_episode_uri,audiobook_title,audiobook_uri,audiobook_chapter_uri,audiobook_chapter_title,reason_start,reason_end,shuffle,skipped,offline,offline_timestamp,incognito_mode
0,2018-07-30T15:39:56Z,"Android OS 5.1 API 22 (LENOVO, Lenovo A7010a48)",11648,HU,176.63.30.118,None,None,None,None,#0 - Pilot,The Misfits Podcast,spotify:episode:67vcPiq854ZJhZUjEobsL0,None,None,None,None,clickrow,unexpected-exit-while-paused,False,False,False,NaN,False
1,2018-07-31T11:57:58Z,"Android OS 5.1 API 22 (LENOVO, Lenovo A7010a48)",50665,HU,176.63.30.166,None,None,None,None,#0 - Pilot,The Misfits Podcast,spotify:episode:67vcPiq854ZJhZUjEobsL0,None,None,None,None,appload,endplay,False,False,False,NaN,False
2,2018-07-31T13:18:15Z,"Android OS 5.1 API 22 (LENOVO, Lenovo A7010a48)",1133551,HU,37.76.106.188,None,None,None,None,#1 - The Poland Story,The Misfits Podcast,spotify:episode:0FqoVwr8OOHVRTUkm2Q5mL,None,None,None,None,clickrow,unexpected-exit-while-paused,False,False,False,NaN,False


In [99]:
# 3) Minimális előkészítés
df = df.copy()  # Másolat készítése, hogy az eredeti DataFrame érintetlen maradjon
df['ts'] = pd.to_datetime(df.get('ts'), utc=True, errors='coerce')  # Időbélyeg datetime-á alakítása UTC-ben; hibás érték -> NaT
df = df.dropna(subset=['ts'])  # Hibás/Nem értelmezhető időbélyegek eldobása

df['sec'] = (df.get('ms_played').fillna(0) / 1000).astype(float)  # Ezredmásodperc -> másodperc; hiányzó értékek 0-ra töltése
df = df[df['sec'] >= 0]  # Negatív időtartamok kiszűrése (adattisztítás)
df['min'] = df['sec'] / 60.0  # Másodperc -> perc (könnyebb aggregálni)

df['artist'] = df.get('master_metadata_album_artist_name')  # Előadó neve kényelmi oszlopba
df['track']  = df.get('master_metadata_track_name')  # Dal címe kényelmi oszlopba

df['date']  = df['ts'].dt.date  # Csak a dátum része (YYYY-MM-DD)
df['year']  = df['ts'].dt.year  # Év (YYYY)
df['month'] = df['ts'].dt.to_period('M').astype(str)  # Hónap (YYYY-MM)
df['hour']  = df['ts'].dt.hour  # Óra (0..23)
df['dow']   = df['ts'].dt.dayofweek  # Hét napja (0=hétfő .. 6=vasárnap)

def simplify_platform(s: str) -> str:  # Platformok egyszerűsítése közös kategóriákra
    if not isinstance(s, str):  # Ha nem szöveg (NaN, None, stb.)
        return 'Other'  # Egyéb kategória
    s = s.lower()  # Kisbetűsítés az összehasonlításhoz
    if 'android' in s: return 'Android'  # Android eszköz
    if 'ios' in s or 'iphone' in s or 'ipad' in s: return 'iOS'  # Apple mobil eszköz
    if 'windows' in s: return 'Windows'  # Windows asztali/kliens
    if 'mac os' in s or 'macos' in s or 'os x' in s: return 'Mac'  # macOS/OS X
    if 'linux' in s: return 'Linux'  # Linux
    return 'Other'  # Egyéb (nem azonosított)

df['platform_simple'] = df.get('platform').apply(simplify_platform)  # Egyszerűsített platform hozzárendelése
df['offline'] = df.get('offline', False).fillna(False).astype(bool)  # Offline jelző normalizálása logikaivá
df['skipped'] = df.get('skipped', False).fillna(False).astype(bool)  # Átugrás jelző normalizálása logikaivá

print(df.shape)  # DataFrame mérete (sor, oszlop)
display(df[['ts','artist','track','min','platform_simple','offline']].head(5))  # Ellenőrző minta megjelenítése

c:\Users\gondo\anaconda3\lib\site-packages\pandas\core\arrays\datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(


(134514, 33)


,ts,artist,track,min,platform_simple,offline
0,2018-07-30 15:39:56+00:00,None,None,0.194133,Android,False
1,2018-07-31 11:57:58+00:00,None,None,0.844417,Android,False
2,2018-07-31 13:18:15+00:00,None,None,18.892517,Android,False
3,2018-07-31 15:39:00+00:00,None,None,18.892517,Android,False
4,2018-08-01 10:33:50+00:00,None,None,2.998250,Android,False


In [100]:
# 4) Összes hallgatási idő – év és hónap szerint
ev = (
    df.groupby('year')['min'].sum()  # Év szerinti csoportosítás, percek összeadása
    .reset_index()  # Visszaalakítás táblává (index -> oszlop)
    .rename(columns={'min': 'Összes perc/év'})  # Beszédes oszlopnév
    .round(1)  # Kerekítés 1 tizedesre
)

honap = (
    df.groupby('month')['min'].sum()  # Hónap szerinti csoportosítás, percek összeadása
    .reset_index()  # Táblává alakítás
    .rename(columns={'min': 'Összes perc/hó'})  # Beszédes oszlopnév
    .sort_values('month')  # Időrendi rendezés (YYYY-MM)
    .round(1)  # Kerekítés 1 tizedesre
)

print('Éves összesítés:')
display(ev)  # Éves összes percek táblázata

print('Havi összesítés (időrendi):')
display(honap.tail(24))  # Utolsó 24 hónap (friss időszak fókusz)

Éves összesítés:


,year,Összes perc/év
0,2018,343.7
1,2019,317.9
2,2020,1202.4
3,2021,8397.5
4,2022,37913.3
5,2023,49643.0
6,2024,72056.1
7,2025,50972.2


Havi összesítés (időrendi):


,month,Összes perc/hó
39,2023-10,4130.0
40,2023-11,3449.7
41,2023-12,5211.1
42,2024-01,8572.2
43,2024-02,5159.8
44,2024-03,4924.9
45,2024-04,6252.1
46,2024-05,5764.1
47,2024-06,6752.0
48,2024-07,6890.1


In [101]:
# 5/a) Top 10 előadó (összes perc alapján)
top_artists = (
    df.dropna(subset=['artist'])  # Hiányzó előadónevek eldobása
    
    .groupby('artist')['min'].sum()  # Előadónként percek összege
    
    .reset_index()  # Táblává alakítás (index -> oszlop)
    
    .rename(columns={'min': 'Összes perc'})  # Beszédes oszlopnév
    
    .sort_values('Összes perc', ascending=False)  # Csökkenő sorrend
    
    .head(10)  # Top 10
    
    .round({'Összes perc': 1})  # Kerekítés 1 tizedesre
)
display(top_artists)  # Top 10 előadó percekkel

# 5/b) Top 10 dal (lejátszások száma alapján)
top_tracks = (
    df.dropna(subset=['artist','track'])  # Hiányzó előadó/dal sorok eldobása
    
    .groupby(['artist','track']).size()  # Előadó–dal páronként előfordulásszám
    
    .reset_index(name='Lejátszások száma')  # Számláló oszlop elnevezése
    
    .sort_values('Lejátszások száma', ascending=False)  # Csökkenő sorrend
    
    .head(10)  # Top 10 dal
)
display(top_tracks)  # Top 10 dal az indítások számával

,artist,Összes perc
719,Eminem,22989.2
1554,NF,18776.3
1878,Ryan Caraveo,8406.3
2493,mgk,7741.6
1145,Kanye West,5908.0
1164,Kendrick Lamar,5584.4
1103,Joyner Lucas,5318.7
2338,Witt Lowry,4330.7
1226,Krúbi,3917.0
306,Beton.Hofi,3866.8


,artist,track,Lejátszások száma
2353,Eminem,Lucky You (feat. Joyner Lucas),382
2439,Eminem,Till I Collapse,360
7568,XXXTENTACION,SAD!,336
5244,NF,Nate,306
5203,NF,DRIFTING,305
5232,NF,Let You Down,298
2061,EDEN,Wake Up,292
5260,NF,Remember This,286
2372,Eminem,No Regrets (feat. Don Toliver),278
6199,Ryan Caraveo,From Hell With Love,274


In [102]:
# 7) Platform-megoszlás – összes perc és részesedés (%)
plat = (
    df.groupby('platform_simple')['min'].sum().reset_index()  # Platformonként percek összege + táblává alakítás
    .rename(columns={'min': 'Összes perc'})  # Beszédes oszlopnév
    .sort_values('Összes perc', ascending=False)  # Csökkenő sorrend
)

osszes = plat['Összes perc'].sum()  # Teljes perc (nevező)

plat['Részesedés (%)'] = (
    100 * plat['Összes perc'] / (osszes if osszes else np.nan)  # Százalékos részesedés (0 osztás védve)
)

display(plat.round({'Összes perc': 1, 'Részesedés (%)': 2}))  # Kerekítés + megjelenítés

,platform_simple,Összes perc,Részesedés (%)
2,Windows,142936.6,64.72
0,Android,77802.6,35.23
1,Other,106.9,0.05


In [103]:
# 8) Top 10 nap – legtöbb hallgatott perc + aktív napok száma
napok = (
    df.groupby('date')['min'].sum().reset_index()  # Naponként percek összege + táblává alakítás
    .rename(columns={'min': 'Összes perc'})  # Beszédes oszlopnév
    .sort_values('Összes perc', ascending=False)  # Csökkenő sorrend (legerősebb napok felül)
)

print('Top 10 nap (legtöbb hallgatott perc):')
display(
    napok.head(10).round({'Összes perc': 1})  # Top 10 nap kerekítve
)

aktiv_napok = napok['date'].nunique()  # Egyedi napok száma (aktív napok)
print(f'Összes aktív nap száma: {aktiv_napok:,}')  # Aktív napok száma formázva

Top 10 nap (legtöbb hallgatott perc):


,date,Összes perc
1094,2024-08-05,878.5
893,2024-01-17,621.4
894,2024-01-18,564.3
1419,2025-06-29,561.8
1101,2024-08-12,557.7
547,2023-01-29,553.3
1028,2024-05-31,548.0
1413,2025-06-23,547.4
896,2024-01-20,524.1
1451,2025-07-31,509.2


Összes aktív nap száma: 1,504


In [104]:
# 9) Havi előadói diverzitás – egyedi előadók száma hónaponként
div = (
    df.dropna(subset=['artist'])  # Hiányzó előadónév eldobása
    .groupby('month')['artist'].nunique()  # Hónaponként egyedi előadók száma
    .reset_index(name='Egyedi előadók száma')  # Táblává alakítás és oszlopnév
    .sort_values('month')  # Időrendi rendezés
)

display(div.tail(24))  # Utolsó 24 hónap megjelenítése

,month,Egyedi előadók száma
34,2023-10,213
35,2023-11,227
36,2023-12,371
37,2024-01,381
38,2024-02,255
39,2024-03,228
40,2024-04,346
41,2024-05,316
42,2024-06,253
43,2024-07,260


In [105]:
# 10) Hosszú lejátszások aránya (≥ 2 perc) – összesen és havi bontásban
kuszob_perc = 2.0  # Hosszú lejátszás küszöbe percben

_tmp = df.assign(  # Ideiglenes DF hosszú-jelzővel
    hosszu=lambda x: x['min'] >= kuszob_perc  # True, ha eléri a küszöböt
)

lejatszas_db = int(len(_tmp))  # Összes lejátszás száma (sorok száma)
hosszu_db = int(_tmp['hosszu'].sum())  # Hosszú lejátszások száma (True értékek összege)

ossz_arany = (
    100 * hosszu_db / lejatszas_db if lejatszas_db else np.nan  # Összes arány % (0 osztás védve)
)
print(
    f'Összes hosszú lejátszás aránya (≥ {kuszob_perc} perc): {ossz_arany:.1f}%'
)

havi = (
    _tmp.assign(Ev_es_honap=_tmp['ts'].dt.to_period('M').astype(str))  # YYYY-MM kulcs
    .groupby('Ev_es_honap')  # Hónaponkénti csoportosítás
    .agg(  # Named aggregation: több metrika egyszerre
        lejatszas_db=('hosszu','size'),  # Mintaszám
        hosszu_db=('hosszu','sum'),     # Hosszú darabszám
    )
    .assign(
        hosszu_szazalek=lambda t: 100 * t['hosszu_db'] / t['lejatszas_db']  # Havi %
    )
    .reset_index()  # Táblává alakítás
    .sort_values('Ev_es_honap')  # Időrendi rendezés
)

display(havi.tail(24).round({'hosszu_szazalek': 1}))  # Utolsó 24 hónap megjelenítése

Összes hosszú lejátszás aránya (≥ 2.0 perc): 41.5%


c:\Users\gondo\anaconda3\lib\site-packages\pandas\core\arrays\datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(


,Ev_es_honap,lejatszas_db,hosszu_db,hosszu_szazalek
39,2023-10,2311,1064,46.0
40,2023-11,2297,910,39.6
41,2023-12,3231,1363,42.2
42,2024-01,3663,2226,60.8
43,2024-02,2336,1301,55.7
44,2024-03,2397,1260,52.6
45,2024-04,3815,1688,44.2
46,2024-05,3281,1507,45.9
47,2024-06,3055,1799,58.9
48,2024-07,3156,1844,58.4


In [ ]:
# 11) Óránkénti megoszlás – melyik órákban hallgatsz legtöbbet
ora = (
    df.groupby('hour')['min'].sum().reset_index()  # Óránként percek összege + táblává alakítás
    .rename(columns={'min':'Összes perc'})  # Beszédes oszlopnév
)

ora = ora.sort_values('hour')  # Időrendi rendezés (0..23)
q
display(ora)  # Teljes táblázat

print('Top 5 óra a legtöbb perccel:') 
display(
    ora.sort_values('Összes perc', ascending=False)
       .head(5)  # Top 5 óra
       .round({'Összes perc':1})  # Kerekítés 1 tizedesre
)

,hour,Összes perc
0,0,10719.647817
1,1,8506.656350
2,2,5344.092100
3,3,2422.936333
4,4,1011.945917
5,5,1877.169633
6,6,2309.650000
7,7,2534.519617
8,8,3701.609750
9,9,5535.665000


Top 5 óra a legtöbb perccel:


,hour,Összes perc
17,17,14841.7
16,16,14170.8
13,13,13542.9
15,15,13500.8
14,14,13416.0


In [ ]:
# 12) Napok szerinti megoszlás – hét mely napjain hallgatsz legtöbbet
nap = (
    df.groupby('dow')['min'].sum().reset_index()  # Hét napja szerint percek összege + táblává alakítás
    .rename(columns={'min':'Összes perc'})  # Beszédes oszlopnév
)

nap = nap.sort_values('dow')  # Hétfőtől vasárnapig

nap['Nap'] = nap['dow'].map({  # Magyar napnevek
    0:'Hétfő', 1:'Kedd', 2:'Szerda', 3:'Csütörtök', 4:'Péntek', 5:'Szombat', 6:'Vasárnap'
})

nap = nap[['Nap','Összes perc']]  # Tiszta táblázat (csak felirat és érték)

display(nap)  # Megjelenítés

print('Top 3 nap:')  # Top napok
display(
    nap.sort_values('Összes perc', ascending=False) # Csökkenő sorrend
       .head(3)
       .round({'Összes perc':1})  # Top 3 nap kerekítve
)

,Nap,Összes perc
0,Hétfő,35123.045700
1,Kedd,28982.355683
2,Szerda,31394.055383
3,Csütörtök,31687.185217
4,Péntek,35217.164167
5,Szombat,29799.511967
6,Vasárnap,28642.823433


Top 3 nap:


,Nap,Összes perc
4,Péntek,35217.2
0,Hétfő,35123.0
3,Csütörtök,31687.2


In [108]:
# 13) Átugrási ráta előadónként (mintaszűrés mellett)
sk = df.copy()  # Másolat készítése

sk['skipped'] = (
    sk.get('skipped', False).fillna(False).astype(bool)  # Átugrás jelző normalizálása logikaivá (NaN->False)
)

stats = (
    sk.dropna(subset=['artist'])  # Hiányzó előadónevek eldobása
    .groupby('artist')  # Előadónkénti csoportosítás
    .agg(  # Named aggregation
        minta=('skipped','size'),   # Mintaszám
        kihagyas=('skipped','sum'), # Átugrások száma
        perc=('min','sum'),         # Összes perc
    )
    .reset_index()  # Táblává alakítás
)

stats = stats[stats['minta'] >= 150].copy()  # Mintaküszöb (stabilabb arány)

stats['Átugrási ráta (%)'] = (
    100 * stats['kihagyas'] / stats['minta']  # Százalékos ráta számítása
)

print('Legmagasabb átugrási ráta – top 10:')
display(
    stats.sort_values('Átugrási ráta (%)', ascending=False)
         .head(10)
         .round({'Átugrási ráta (%)':2, 'perc':1})  # Top 10 magas ráta
)

print('\nLegalacsonyabb átugrási ráta – top 10:')
display(
    stats.sort_values('Átugrási ráta (%)', ascending=True)
         .head(10)
         .round({'Átugrási ráta (%)':2, 'perc':1})  # Top 10 alacsony ráta
)

Legmagasabb átugrási ráta – top 10:


,artist,minta,kihagyas,perc,Átugrási ráta (%)
77,AWS,282,214,375.5,75.89
1290,Lil Frakk,294,222,338.8,75.51
2464,gnash,167,121,212.7,72.46
1226,Krúbi,2037,1462,3917.0,71.77
230,BEATó,442,311,456.3,70.36
675,Dé:Nash,306,214,533.5,69.93
797,Frank Ocean,248,173,323.2,69.76
1535,Munn,175,118,231.3,67.43
469,Chris Brown,191,127,237.4,66.49
808,Future,176,115,197.7,65.34



Legalacsonyabb átugrási ráta – top 10:


,artist,minta,kihagyas,perc,Átugrási ráta (%)
1266,League of Legends,160,21,199.0,13.12
1997,Skylar Grey,272,62,640.0,22.79
336,Bo Burnham,815,209,1728.2,25.64
1398,Mackenzy Mackay,162,43,294.0,26.54
1308,Lilly Wood and The Prick,236,70,292.0,29.66
690,Ed Sheeran,522,177,689.9,33.91
1665,OneRepublic,211,73,377.6,34.60
816,GAYLE,210,73,257.6,34.76
1742,Polo G,719,251,1150.1,34.91
1277,Lewis Capaldi,164,58,336.4,35.37
